# A whole workflow in production environment after my forecaster is developed

In model developing process, `TSDataset` is used to preprocess(including feature engineering, data sampling, scaling, ...) the raw data the postprocess the predicted result(majorly unscaling). This post provides a way by which users could replay the preprocessing and postprocessing in production environment(e.g. model serving).

In this guide, we will

1. Train a TCNForecaster with nyc_taxi dataset and export the model in onnx type and save the scaler.
2. Show users how to replay the preprocessing and postprocessing in production environment.
3. Evaluate the performance of preprocessing and postprocessing
4. More tips about this topic.

## Forecaster developing

First let's prepare the data. We will manually download the data to show the details.

In [15]:
# run following
!wget https://raw.githubusercontent.com/numenta/NAB/v1.0/data/realKnownCause/nyc_taxi.csv

--2022-12-22 16:25:26--  https://raw.githubusercontent.com/numenta/NAB/v1.0/data/realKnownCause/nyc_taxi.csv
Resolving proxy-prc.intel.com (proxy-prc.intel.com)... 10.240.252.16
Connecting to proxy-prc.intel.com (proxy-prc.intel.com)|10.240.252.16|:913... connected.
Proxy request sent, awaiting response... 200 OK
Length: 265771 (260K) [text/plain]
Saving to: ‘nyc_taxi.csv.1’

nyc_taxi.csv.1      100%[===================>] 259.54K   362KB/s    in 0.7s    

2022-12-22 16:25:27 (362 KB/s) - ‘nyc_taxi.csv.1’ saved [265771/265771]



Then we may load the data to pandas dataframe and carry out preprocessing through `TSDataset`. You could refer to

- [How to preprocess my own data](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_preprocess_my_data.html)
- [How to sample data through sliding window]()

for details.

In [1]:
from sklearn.preprocessing import StandardScaler
from bigdl.chronos.data import TSDataset, get_public_dataset
import pandas as pd

# load the data to pandas dataframe
df = pd.read_csv("nyc_taxi.csv", parse_dates=["timestamp"])

# use nyc_taxi public dataset
train_data, _, test_data = TSDataset.from_pandas(df,
                                                 dt_col="timestamp",
                                                 target_col="value",
                                                 repair=False,
                                                 with_split=True,
                                                 test_ratio=0.1)

# create a scaler for data scaling
scaler = StandardScaler()

# preprocess(generate datetime feature, scale and roll samping)
for data in [train_data, test_data]:
    data.gen_dt_feature(features=["WEEKDAY", "HOUR", "MINUTES"])\
        .scale(scaler, fit=(data is train_data))\
        .roll(lookback=48, horizon=24)

Developing a forecaster on this data is quite easy. You may refer to other how-to guide for more detailed information.

- [How to create a Forecaster](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_create_forecaster.html)
- [Train forcaster on single node](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_train_forecaster_on_one_node.html)

In [2]:
from bigdl.chronos.forecaster import TCNForecaster  # TCN is algorithm name

# create a forecaster
forecaster = TCNForecaster.from_tsdataset(train_data)

# train the forecaster
forecaster.fit(train_data)

/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 2566135082
Global seed set to 2566135082
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type             | Params
-------------------------------------------
0 | model | NormalizeTSModel | 5.6 K 
1 | loss  | MSELoss          | 0     
-------------------------------------------
5.6 K     Trainable params
0         Non-trainable params
5.6 K     Total params
0.022     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 289/289 [00:02<00:00, 114.66it/s, loss=0.266]


`Forecaster` API is for quick iteration during the development, when a forecaster is developed with satisfying accuracy and performance, users may prefer to export the model to formats that are easier to deploy in production environment (e.g., ONNX, openVINO, torchscript, ...). We choose to use ONNX here as an example. You may refer to other how to guides for more details.

- [Export the ONNX model files to disk](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_export_onnx_files.html)
- [Export the OpenVINO model files to disk](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_export_openvino_files.html)
- [Export the TorchScript model files to disk](https://bigdl.readthedocs.io/en/latest/doc/Chronos/Howto/how_to_export_torchscript_files.html)

In [3]:
# save the forecaster in onnx type
forecaster.export_onnx_file(dirname="nyc_taxi_onnx_model", quantized_dirname=None)

/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py:1487: UserWarning: The environment variable OMP_NUM_THREADS is suggested to be same as thread_num.You can use 'export OMP_NUM_THREADS=1'.
  warnings.warn("The environment variable OMP_NUM_THREADS is suggested to be same "


In [4]:
from pathlib import Path
module_dir = Path("forecaster")
module_dir.mkdir(exist_ok=True)
forecaster.export_torchscript_module(train_data, path_dir="forecaster")

module_dir = Path("forecaster_ts")
module_dir.mkdir(exist_ok=True)
forecaster.export_torchscript_file(dirname=module_dir)

/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py:1567: UserWarning: The environment variable OMP_NUM_THREADS is suggested to be same as thread_num.You can use 'export OMP_NUM_THREADS=1'.
  warnings.warn("The environment variable OMP_NUM_THREADS is suggested to be same "


If a scaler is used during the preprocessing process, then it should be saved for production environment.

In [5]:
import pickle

# save the scaler
# There are many ways, we use pickle here
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

## In production environment
When you have successfully developed a model your are really satisfied with, then it's time to deploy your model. With the highly optimized model you just exported, the whole process could be suprising easy.

There are 2 possibilities for deployment:
1. You will use the model in a monolithic application, where the input, model inference and output is located on the same server.
2. You will use a server-client model, where you may want to adopt some model serving tools (e.g., torchserve, OpenVINO server, Triton, ...). This means that users will seperate model inference with other workload.

For the first choice, you may directly call some inference engine API (e.g., onnxruntime, OpenVINO, ...) in your application. For the second choice, this may depends on different model serving tools' procedure. We have [an example to serve a forecaster on torchserve](https://github.com/intel-analytics/BigDL/tree/main/python/chronos/example/serving).

For both choicies, it's common to have a single sample data to come, here we use the last sample of nyc taxi dataset as example

In [6]:
# generate data to predict in a local csv file
_, _, test_data = get_public_dataset("nyc_taxi")
test_data.df[-48:].to_csv("inference_data.csv", index=False)

Then we could load all the items we need, this includes the scaler and onnx file we just dumped, and the data to be inferenced.

In [7]:
import onnxruntime

# load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# load the onnx file to onnxruntime
session = onnxruntime.InferenceSession("nyc_taxi_onnx_model/onnx_saved_model.onnx")

# load the data to be predicted
df = pd.read_csv("inference_data.csv", parse_dates=["timestamp"])

The preprocess process should be the same as how you processed your data when developing the forecaster, except

> 📝**Note**
> 
> There are 2 exceptions here that should be followed carefully
>
> - Please make sure to set `deploy_mode=True` when creating TSDataset through `TSDataset.from_pandas`, `TSDataset.from_parquet` and `TSDataset.from_prometheus`, which will reduce data processing latency and set necessary parameters for deployment.
> 
> - For `scale`, please make sure using the scaler you dumped and loaded back.

In [8]:
def preprocess_during_deployment(df, scaler):
    tsdata = TSDataset.from_pandas(df,
                                   dt_col="timestamp",
                                   target_col="value",
                                   repair=False,
                                   deploy_mode=True)
    tsdata.gen_dt_feature(features=["WEEKDAY", "HOUR", "MINUTES"])\
          .scale(scaler)\
          .roll(lookback=48, horizon=24)
    data = tsdata.to_numpy()
    return tsdata, data

For postprocessing, if scaler is used, then `unscale_numpy` is needed.

In [9]:
def postprocess_during_deployment(data, tsdata):
    return tsdata.unscale_numpy(data)

They users could predict the data easily by a clear process.

In [10]:
tsdata, data = preprocess_during_deployment(df, scaler)
data = session.run(None, {'x': data})[0]
processed_data = postprocess_during_deployment(data, tsdata)


In [11]:
import torch
tsdata, data = preprocess_during_deployment(df, scaler)
input_tensor = torch.from_numpy(data)
module = torch.jit.load(Path("forecaster_ts") / "ckpt.pth")
data = module.forward(input_tensor)
processed_data_1 = postprocess_during_deployment(data.numpy(), tsdata)

In [15]:
import torch
tsdata, data = preprocess_during_deployment(df, scaler)
print(data.dtype)
input_tensor = torch.from_numpy(data)
module = torch.jit.load(Path("forecaster_ts") / "ckpt.pth")
data = module.forward(input_tensor)
processed_data_2 = postprocess_during_deployment(data.numpy(), tsdata)

KeyboardInterrupt: 

Exception ignored in: '_pydevd_frame_eval.pydevd_frame_evaluator.get_bytecode_while_frame_eval_38'
Traceback (most recent call last):
  File "_pydevd_frame_eval/pydevd_frame_evaluator.pyx", line 258, in _pydevd_frame_eval.pydevd_frame_evaluator.get_func_code_info
  File "/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd_file_utils.py", line 885, in get_abs_path_real_path_and_base_from_frame
    ret = get_abs_path_real_path_and_base_from_file(f)
  File "/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd_file_utils.py", line 852, in get_abs_path_real_path_and_base_from_file
    abs_path, canonical_normalized_filename = _abs_and_canonical_path(f)
  File "/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd_file_utils.py", line 411, in _abs_and_canonical_path
    real_path = _apply_func_and_normalize_case(filename, os_path_real_path, isa

SystemError: <class 'pandas.core.groupby.ops.BaseGrouper'> returned NULL without setting an error

In [13]:
from numpy.testing import assert_array_almost_equal

assert_array_almost_equal(processed_data_1, processed_data_2)

In [14]:
import torch
import os
import numpy as np

deployment_df = pd.read_csv("inference_data.csv", parse_dates=["timestamp"])
tsdata_test = TSDataset.from_pandas(deployment_df,
                                dt_col="timestamp",
                                target_col="value",
                                repair=False,
                                deploy_mode=True)
tsdata_test.gen_dt_feature(features=["WEEKDAY", "HOUR", "MINUTES"])
# drop datetime col since it can't be converted to Pytorch tensor
tsdata_test.df.drop(columns=tsdata_test.dt_col, inplace=True)
tsdata_test.df["id"] = np.array([0] * len(tsdata_test.df))
input_tensor = torch.from_numpy(tsdata_test.df.values).type(torch.float64)

jit_path = os.path.join("forecaster", "forecaster_pipeline.pt")
jit_module = torch.jit.load(jit_path)

output = jit_module.forward(input_tensor)

7


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/bigdl/chronos/forecaster/utils.py", line 12, in forward
    data: Tensor) -> Tensor:
    preprocess = self.preprocess
    preprocess_output = (preprocess).forward(data, )
                         ~~~~~~~~~~~~~~~~~~~ <--- HERE
    inference = self.inference
    inference_output = (inference).forward(preprocess_output, )
  File "code/__torch__/bigdl/chronos/data/utils/export_torchscript.py", line 20, in forward
    _0 = torch.eq(operation, "preprocessing")
    if _0:
      _2 = (self).export_preprocessing(data, )
            ~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
      _1 = _2
    else:
  File "code/__torch__/bigdl/chronos/data/utils/export_torchscript.py", line 37, in export_preprocessing
    _9 = torch.slice(data)
    _10 = annotate(List[Optional[Tensor]], [None, _8])
    _11 = torch.index_put_(_9, _10, _7)
          ~~~~~~

In [18]:
from numpy.testing import assert_array_almost_equal
print(output.numpy().dtype)
print(processed_data_2.dtype)
assert_array_almost_equal(processed_data_2, output.numpy())

float64


NameError: name 'output' is not defined

In [19]:
print("""
{
	"name": "RuntimeError",
	"message": "The following operation failed in the TorchScript interpreter.\nTraceback of TorchScript, serialized code (most recent call last):\n  File \"code/__torch__/bigdl/chronos/forecaster/utils.py\", line 14, in forward\n    preprocess_output = (preprocess).forward(data, )\n    inference = self.inference\n    inference_output = (inference).forward(preprocess_output, )\n                        ~~~~~~~~~~~~~~~~~~ <--- HERE\n    postprocess = self.postprocess\n    postprocess_output = (postprocess).forward(inference_output, )\n  File \"code/__torch__/bigdl/nano/pytorch/lightning/___torch_mangle_88.py\", line 11, in forward\n    x0 = torch.sub(x, seq_last)\n    input = torch.permute(x0, [0, 2, 1])\n    input0 = torch._convolution(input, CONSTANTS.c0, CONSTANTS.c1, [1], [2], [1], False, [0], 1, False, False, True, True)\n             ~~~~~~~~~~~~~~~~~~ <--- HERE\n    input1 = torch.relu(input0)\n    _2 = torch.slice(input1, 0, 0, 9223372036854775807)\n\nTraceback of TorchScript, original code (most recent call last):\n  File \"/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/utils.py\", line 194, in forward\n    def forward(self, data):\n        preprocess_output = self.preprocess.forward(data)\n        inference_output = self.inference.forward(preprocess_output)\n                           ~~~~~~~~~~~~~~~~~~~~~~ <--- HERE\n        postprocess_output = self.postprocess(inference_output)\n        return postprocess_output\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/conv.py(304): _conv_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/conv.py(307): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/container.py(139): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/model/tcn.py(100): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/container.py(139): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/model/tcn.py(142): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/pytorch/model_wrapper/normalization.py(35): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/pytorch/lightning.py(99): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/jit/_trace.py(974): trace_module\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/jit/_trace.py(759): trace\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/deps/ipex/ipex_inference_model.py(112): __init__\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/deps/ipex/ipex_api.py(62): PytorchIPEXJITModel\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/pytorch/inference/optimizer.py(918): trace\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1576): build_jit\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1653): export_torchscript_file\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1662): export_torchscript_module\n/tmp/ipykernel_28184/2975897614.py(4): <module>\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3553): run_code\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3473): run_ast_nodes\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3258): run_cell_async\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/async_helpers.py(78): _pseudo_sync_runner\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3030): _run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(2976): run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/zmqshell.py(528): run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/ipkernel.py(387): do_execute\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(730): execute_request\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(406): dispatch_shell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(499): process_one\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(510): dispatch_queue\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/events.py(88): _run\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/base_events.py(1786): _run_once\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/base_events.py(541): run_forever\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/tornado/platform/asyncio.py(215): start\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelapp.py(712): start\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/traitlets/config/application.py(982): launch_instance\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel_launcher.py(17): <module>\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/runpy.py(85): _run_code\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/runpy.py(193): _run_module_as_main\nRuntimeError: expected scalar type Double but found Float\n",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mRuntimeError\u001b[0m                              Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_28184/1269694426.py\u001b[0m in \u001b[0;36m<module>\u001b[0;34m\u001b[0m\n\u001b[1;32m     18\u001b[0m \u001b[0mjit_module\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mtorch\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mjit\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mload\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mjit_path\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     19\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 20\u001b[0;31m \u001b[0moutput\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mjit_module\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mforward\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0minput_tensor\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;31mRuntimeError\u001b[0m: The following operation failed in the TorchScript interpreter.\nTraceback of TorchScript, serialized code (most recent call last):\n  File \"code/__torch__/bigdl/chronos/forecaster/utils.py\", line 14, in forward\n    preprocess_output = (preprocess).forward(data, )\n    inference = self.inference\n    inference_output = (inference).forward(preprocess_output, )\n                        ~~~~~~~~~~~~~~~~~~ <--- HERE\n    postprocess = self.postprocess\n    postprocess_output = (postprocess).forward(inference_output, )\n  File \"code/__torch__/bigdl/nano/pytorch/lightning/___torch_mangle_88.py\", line 11, in forward\n    x0 = torch.sub(x, seq_last)\n    input = torch.permute(x0, [0, 2, 1])\n    input0 = torch._convolution(input, CONSTANTS.c0, CONSTANTS.c1, [1], [2], [1], False, [0], 1, False, False, True, True)\n             ~~~~~~~~~~~~~~~~~~ <--- HERE\n    input1 = torch.relu(input0)\n    _2 = torch.slice(input1, 0, 0, 9223372036854775807)\n\nTraceback of TorchScript, original code (most recent call last):\n  File \"/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/utils.py\", line 194, in forward\n    def forward(self, data):\n        preprocess_output = self.preprocess.forward(data)\n        inference_output = self.inference.forward(preprocess_output)\n                           ~~~~~~~~~~~~~~~~~~~~~~ <--- HERE\n        postprocess_output = self.postprocess(inference_output)\n        return postprocess_output\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/conv.py(304): _conv_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/conv.py(307): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/container.py(139): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/model/tcn.py(100): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/container.py(139): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/model/tcn.py(142): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/pytorch/model_wrapper/normalization.py(35): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/pytorch/lightning.py(99): forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1118): _slow_forward\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/nn/modules/module.py(1130): _call_impl\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/jit/_trace.py(974): trace_module\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/torch/jit/_trace.py(759): trace\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/deps/ipex/ipex_inference_model.py(112): __init__\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/deps/ipex/ipex_api.py(62): PytorchIPEXJITModel\n/home/zhaojieh/BigDL-hzj/python/nano/src/bigdl/nano/pytorch/inference/optimizer.py(918): trace\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1576): build_jit\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1653): export_torchscript_file\n/home/zhaojieh/BigDL-hzj/python/chronos/src/bigdl/chronos/forecaster/base_forecaster.py(1662): export_torchscript_module\n/tmp/ipykernel_28184/2975897614.py(4): <module>\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3553): run_code\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3473): run_ast_nodes\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3258): run_cell_async\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/async_helpers.py(78): _pseudo_sync_runner\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3030): _run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py(2976): run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/zmqshell.py(528): run_cell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/ipkernel.py(387): do_execute\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(730): execute_request\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(406): dispatch_shell\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(499): process_one\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelbase.py(510): dispatch_queue\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/events.py(88): _run\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/base_events.py(1786): _run_once\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/asyncio/base_events.py(541): run_forever\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/tornado/platform/asyncio.py(215): start\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel/kernelapp.py(712): start\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/traitlets/config/application.py(982): launch_instance\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/site-packages/ipykernel_launcher.py(17): <module>\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/runpy.py(85): _run_code\n/home/zhaojieh/miniconda3/envs/chronos-dev/lib/python3.7/runpy.py(193): _run_module_as_main\nRuntimeError: expected scalar type Double but found Float\n"
}
    """)


{
	"name": "RuntimeError",
	"message": "The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/bigdl/chronos/forecaster/utils.py", line 14, in forward
    preprocess_output = (preprocess).forward(data, )
    inference = self.inference
    inference_output = (inference).forward(preprocess_output, )
                        ~~~~~~~~~~~~~~~~~~ <--- HERE
    postprocess = self.postprocess
    postprocess_output = (postprocess).forward(inference_output, )
  File "code/__torch__/bigdl/nano/pytorch/lightning/___torch_mangle_88.py", line 11, in forward
    x0 = torch.sub(x, seq_last)
    input = torch.permute(x0, [0, 2, 1])
    input0 = torch._convolution(input, CONSTANTS.c0, CONSTANTS.c1, [1], [2], [1], False, [0], 1, False, False, True, True)
             ~~~~~~~~~~~~~~~~~~ <--- HERE
    input1 = torch.relu(input0)
    _2 = torch.slice(input1, 0, 0, 9223372036854775807)

Traceback of TorchScript